In [1]:
#Importamos las librerias que se ocuparan
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
#Cargamos los datasets que ocuparemos
df_generos = pd.read_parquet(r"C:\Users\Elias\Documents\Primer Proyecto Individual\Datasets limpios\generos_totales")
df_games = pd.read_csv(r"C:\Users\Elias\Documents\Primer Proyecto Individual\Datasets limpios\games_clean")
df_games

,id,app_name,genres,developer,tags,release_date,price,cleaned_price
0,761140.0,Lost Summoner Kitty,Action,Kotoshiro,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",2018-01-04,4.99,4.99
1,643980.0,Ironbound,Indie,Secret Level SRL,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",2018-01-04,Free To Play,0.00
2,670290.0,Real Pool 3D - Poolians,Casual,Poolians.com,"['Free to Play', 'Simulation', 'Sports', 'Casu...",2017-07-24,Free to Play,0.00
3,767400.0,弹炸人2222,Action,彼岸领域,"['Action', 'Adventure', 'Casual']",2017-12-07,0.99,0.99
4,773570.0,Log Challenge,Action,NaN,"['Action', 'Indie', 'Casual', 'Sports']",NaN,2.99,2.99
...,...,...,...,...,...,...,...,...
32130,773640.0,Colony On Mars,Casual,"Nikita ""Ghost_RUS""","['Strategy', 'Indie', 'Casual', 'Simulation']",2018-01-04,1.99,1.99
32131,733530.0,LOGistICAL: South Africa,Casual,Sacada,"['Strategy', 'Indie', 'Casual']",2018-01-04,4.99,4.99
32132,610660.0,Russian Roads,Indie,Laush Dmitriy Sergeevich,"['Indie', 'Simulation', 'Racing']",2018-01-04,1.99,1.99
32133,658870.0,EXIT 2 - Directions,Casual,"xropi,stev3ns","['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",2017-09-02,4.99,4.99


In [3]:
print(df_games['id'].dtype)
print(df_generos['id'].dtype)

float64
float64


In [4]:

# Convertir la columna 'id' de enteros a cadenas para unir los DataFrames correctamente
df_games['id'] = df_games['id'].astype(str)
df_generos['id'] = df_generos['id'].astype(str)

In [5]:
# Combinar los DataFrames
df_merged = df_games.merge(df_generos, on='id', how='left')

# Convertir la columna 'release_date' a formato datetime
df_merged['release_date'] = pd.to_datetime(df_merged['release_date'], errors='coerce')

# Extraer el año de la columna de fecha
df_merged['release_year'] = df_merged['release_date'].dt.year

# Escalar los años de lanzamiento
scaler = StandardScaler()
df_merged['release_year_scaled'] = scaler.fit_transform(df_merged[['release_year']])

# Definir las características (features) a utilizar para el sistema de recomendación
features = ['release_year_scaled'] + list(df_generos.columns[1:])  # Usamos la columna 'release_year_scaled' en lugar de 'release_date'

# Seleccionar las columnas finales con las características
df_final = df_merged[['id'] + features]
df_final= df_final.merge(df_games[['id', 'app_name']], on='id', how='left')
# Verificar el resultado
df_final.head()


,id,release_year_scaled,360 Video,Accounting,Action,Adventure,Animation & Modeling,Anime,Arcade,Audio Production,...,Twin Stick Shooter,Utilities,VR,Video Production,Violent,War,Web Publishing,World War II,Zombies,app_name
0,761140.0,0.923811,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lost Summoner Kitty
1,643980.0,0.923811,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Ironbound
2,670290.0,0.638753,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Real Pool 3D - Poolians
3,767400.0,0.638753,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,弹炸人2222
4,773570.0,NaN,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Log Challenge


In [6]:
# Selección aleatoria de 50% de los datos para entrenamiento y 50% para prueba
df_sampled = df_final.sample(frac=0.5, random_state=42)

# Calcular la matriz de similitud entre juegos
similarity_matrix = cosine_similarity(df_sampled[features].fillna(0))
similarity_matrix = np.nan_to_num(similarity_matrix)

In [7]:
def recomendacion_juego_muestreado(game_id, top_n=5):
    # Obtener los IDs de los juegos muestreados
    ids_juegos_muestreados = df_sampled['id'].unique()
    
    # Verificar si el ID del juego está en los juegos muestreados
    if game_id not in ids_juegos_muestreados:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    # Obtener el índice del juego en los datos muestreados
    indice_juego = df_sampled.index[df_sampled['id'] == game_id].tolist()
    
    # Verificar si se encontró el juego en los datos muestreados
    if not indice_juego:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    indice_juego = indice_juego[0]

    # Calcular los puntajes de similitud entre juegos
    puntajes_similitud = list(enumerate(similarity_matrix[indice_juego]))
    puntajes_similitud = sorted(puntajes_similitud, key=lambda x: x[1], reverse=True)

    # Obtener los índices de los juegos similares
    indices_juegos_similares = [i for i, puntaje in puntajes_similitud[1:top_n+1]]
    
    # Obtener los nombres de los juegos similares
    nombres_juegos_similares = df_sampled['app_name'].iloc[indices_juegos_similares].tolist()

    # Mensaje de recomendación
    mensaje_recomendacion = f"Juegos recomendados basados en el ID del juego {game_id} - {df_sampled['app_name'].iloc[indice_juego]}:"
    
    return [mensaje_recomendacion] + nombres_juegos_similares

In [8]:
# Prueba del sistema de recomendación con un ID de juego muestreado
test_game_id = "767400.0"
recommendations = recomendacion_juego_muestreado(test_game_id, top_n=5)

# Mostrar las recomendaciones
print("Recomendaciones para ID de juego", test_game_id)
for rec in recommendations:
    print(rec,end="")

Recomendaciones para ID de juego 767400.0
Juegos recomendados basados en el ID del juego 767400.0 - Fantasy Grounds - Forest Map Pack by Joshua (Map Pack):Oxygen Not IncludedBox Looter 2018Deckbound Heroes (Open Beta)战术狂想1(Chimera of Tactics 1)前程似锦 Excellent Expectations